<a href="https://colab.research.google.com/github/mmsamiei/world/blob/main/world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install Faker

In [37]:
from faker import Faker
import random
import datetime
from itertools import filterfalse
from dateutil import relativedelta


In [38]:
fake = Faker()

In [39]:
class Location:

  def __init__(self, map_size=50):
    self.x =  random.uniform(-map_size, map_size)
    self.y = random.uniform(-map_size, map_size)

  def distance(self, other_location):
    dx = other_location.x - self.x
    dy = other_location.y - self.y
    return (dx**2+dy**2)**0.5

In [40]:
random.choice([70,80,90,100])

70

In [124]:
class Person:
  
  def __init__(self, world):

    self.world = world
    self.gender = random.choices(['M', 'F'])
    self.first_name = fake.first_name_female() if self.gender == 'F' else fake.first_name_male()
    self.last_name = fake.last_name()
    self.birthday = self.world.date
    self.deathday = None
    self.location = Location()
    self.velocity = 10
    self.energy = 1
    self.hunger_thershold = random.choice([0.5,0.6,0.7,0.8,0.9,1])
    self.friends = []

  def __str__(self):
    x = round(self.location.x,2)
    y = round(self.location.y,2)
    age = relativedelta.relativedelta(self.world.date, self.birthday)

    return f"{self.first_name} {self.last_name}  at {x}, {y} energy: {self.energy}  age: {age.years, age.months}"
  

  def age(self):
    age = relativedelta.relativedelta(self.world.date, self.birthday)
    return age.years, age.months, age.days


  def can_eat(self):
    for food in self.world.foods:
      if self.location.distance(food.location) < self.world.distance_can_eat:
        return True, food
    return False, None
  
  def is_hungry(self):
    return self.energy < self.hunger_thershold

  def accept_friendship(self, person):
    return True

  def choose_action(self):
    
    can_eat = self.can_eat()
    if can_eat[0] is True:
      return 'eat', (self, can_eat[1]) 

    if self.is_hungry():
      day_to_death = self.energy / self.world.hunger_penalty
      dx = 0
      dy = 0
      for food in random.sample(self.world.foods[:], len(self.world.foods[:])):
        if self.location.distance(food.location)/self.velocity < day_to_death:
          dx = food.location.x - self.location.x
          dy = food.location.y - self.location.y
          length = (dx**2+dy**2)**0.5
          dx_norm = (dx / length) * (self.velocity ** 0.5)
          dy_norm = (dy / length) * (self.velocity ** 0.5)
          if dx**2 + dy**2 < dx_norm**2 + dy_norm**2:
            dir_x, dir_y = dx, dy
          else:
            dir_x, dir_y= dx_norm, dy_norm
          break
      return 'move', (self, dx, dy)
    

    other_persons = self.world.persons[:]
    other_persons.remove(self)
    for person in random.sample(other_persons, len(other_persons)):
      if self.location.distance(person.location) < 5 and person not in self.friends:
        return 'request_friendship', (self, person)
    
    return None, None

In [125]:
class Food:

  def __init__(self, world):
    self.world = world
    self.location = Location()
    self.produce_date = self.world.date

In [134]:
class World:

  def __init__(self, name):
    self.name = name
    self.persons = []
    self.dead_persons = []
    self.date = datetime.date(2018, 6, 1)
    self.foods = []
    self.hunger_penalty = 0.1
    self.distance_can_eat = 2
    self.every_day_foods = 1
    self.expire_day_foods = 5

  def simulate_day(self):
    self.date += datetime.timedelta(days=1)

    new_person = Person(world=self)
    self.persons.append(new_person)

    for i in range(self.every_day_foods):
      self.foods.append(Food(self))
    self.foods[:] = filterfalse(lambda f: (world.date - f.produce_date).days > self.expire_day_foods , self.foods)

    for person in self.persons[:]:
      self.simulate_person(person)
  

  def simulate_person(self, person):
    person.energy -= self.hunger_penalty
    action, args = person.choose_action()
    
    if action == 'eat':
      food = args[1]
      self.foods.remove(food)
      person.energy = 1
    elif action == 'move':
      dx, dy = args[1], args[2]
      person.location.x += dx
      person.location.y += dy
    elif action == 'request_friendship':
      person1, person2 = args[0], args[1]
      if person2.accept_friendship(person1):
        person1.friends.append(person2)
        person2.friends.append(person1)

    if person.energy <= 0 :
      self.persons.remove(person)
      self.dead_persons.append(person)
      person.deathday = self.date

  def simulate(self, until=1000):
    for i in range(until):
      self.simulate_day()
      #print(len(self.foods))
      print("persons ",len(self.persons), len(self.dead_persons))


In [135]:
world = World('mahdi_world')
world.simulate()

persons  1 0
persons  2 0
persons  3 0
persons  4 0
persons  5 0
persons  6 0
persons  7 0
persons  8 0
persons  9 0
persons  10 0
persons  11 0
persons  11 1
persons  11 2
persons  11 3
persons  12 3
persons  13 3
persons  14 3
persons  15 3
persons  15 4
persons  13 7
persons  14 7
persons  15 7
persons  15 8
persons  14 10
persons  14 11
persons  15 11
persons  14 13
persons  14 14
persons  13 16
persons  14 16
persons  14 17
persons  14 18
persons  14 19
persons  14 20
persons  14 21
persons  14 22
persons  14 23
persons  14 24
persons  14 25
persons  14 26
persons  13 28
persons  13 29
persons  13 30
persons  13 31
persons  14 31
persons  14 32
persons  14 33
persons  14 34
persons  14 35
persons  14 36
persons  14 37
persons  14 38
persons  14 39
persons  14 40
persons  14 41
persons  14 42
persons  13 44
persons  13 45
persons  14 45
persons  14 46
persons  14 47
persons  14 48
persons  15 48
persons  15 49
persons  15 50
persons  15 51
persons  15 52
persons  15 53
persons  15 

In [133]:
for person in world.persons:
  print("\n")
  print(person.first_name, " " , person.last_name, ": ")
  for friend in person.friends:
    print(friend.first_name, " " , friend.last_name)



Matthew   Ross : 


Adam   Russell : 
Steven   Booth
Peter   Burch
Eric   Nash
James   Brock
Jason   Harding


Christopher   Hays : 


Mark   Sanchez : 


Trevor   Sullivan : 


Cameron   Barnett : 


Tyler   Yu : 
Jason   Dixon


Jason   Dixon : 
Tyler   Yu


Patrick   Graham : 
Jason   Harding


Clayton   Mccoy : 


Maxwell   Walters : 


Jorge   Braun : 


Eric   Thomas : 


Joel   Zamora : 


Samuel   Carlson : 
